In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/머신러닝/4장/Santander

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df1 = pd.read_csv('train.csv')
df1.info()

In [ ]:
df1['TARGET'].value_counts()

In [ ]:
df1.describe()

In [ ]:
df1['var3'].value_counts()

In [ ]:
df1['var3'].replace(-999999, 2, inplace = True)
df1.drop('ID', axis = 1, inplace = True)
df1

In [ ]:
cond1 = df1.sum()==0
cond1 = pd.DataFrame(cond1)
drop = cond1[cond1[0]==True].reset_index()['index'].tolist()
print(drop)

In [ ]:
df1.drop(drop, axis = 1, inplace = True)

In [ ]:
df1['TARGET'].value_counts()

In [ ]:
X_df = df1.iloc[:,:-1]
Y_df = df1.iloc[:, -1]

# **SMOTE**

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_df_over, Y_df_over = smote.fit_sample(X_df, Y_df)
print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', X_df.shape, Y_df.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_df_over.shape, Y_df_over.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(Y_df_over).value_counts())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, Y_train, Y_test = train_test_split(X_df_over, Y_df_over, test_size = 0.2, stratify = Y_df_over) # stratify : 분포를 비슷하게 만듦

print(pd.DataFrame(Y_train).value_counts())
print(pd.DataFrame(Y_test).value_counts())

In [ ]:
print(X_train.shape)

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
lgbm_clf = LGBMClassifier(n_estimators=1000)

evals = [(X_test, Y_test)]
lgbm_clf.fit(X_train, Y_train, early_stopping_rounds=100, eval_metric="logloss", eval_set=evals, verbose=True)
pred = lgbm_clf.predict(X_test)

In [ ]:
def clf_eval(Y, pred):
  confusion = confusion_matrix(Y, pred)
  accuracy = accuracy_score(Y, pred)
  precision = precision_score(Y, pred)
  recall = recall_score(Y, pred)
  print('오차행렬 : ','\n',confusion)
  print('acc : {0}, precision : {1}, recall : {2}'.format(accuracy, precision, recall))

In [ ]:
clf_eval(Y_test, pred)

In [ ]:
from lightgbm import plot_importance
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(figsize=(10, 12))
# 사이킷런 래퍼 클래스를 입력해도 무방. 
plot_importance(lgbm_clf, ax=ax, max_num_features=10)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
Xs_train = scaler.transform(X_train)
scaler.fit(X_test)
Xs_test = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(Xs_train, Y_train)
pred = lr.predict(Xs_test)

clf_eval(Y_test, pred)

# **Tensorflow**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential(
    [
        Dense(256, activation="relu"),
        Dense(256, activation="relu"),
        Dropout(0.3),
        Dense(256, activation="relu"),
        Dropout(0.3),
        Dense(1, activation="sigmoid"),
    ]
)

In [ ]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics)

In [ ]:
model.fit(Xs_train, Y_train, batch_size = 2048, epochs = 30, verbose=2, validation_split=0.1)